# Основы сетевого анализа + Создание сетки кварталов + Зональная статистика


## 1.Графы улично-дорожной сети из osmnx


Библиотека osmnx предоставляет удобные инструменты для работы с улично-дорожной сетью как с графом.

Графы в osmnx представляют собой математические структуры, где:

- Узлы (nodes) — это объекты на уличной сети, например, перекрестки или конечные точки дорог.
- Ребра (edges) — это соединения между узлами, то есть сегменты дорог, улиц или других типов инфраструктуры.

В osmnx используется структура графа, где ребра могут содержать информацию о расстояниях, времени в пути, типах дорог и других характеристиках транспортной сети. С помощью этого подхода можно анализировать доступность, проводить сетевой анализ, искать кратчайшие пути, строить зоны доступности и многое другое.


In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

### 1.1 Получение графа уличной сети


Сначала создадим граф для района с использованием данных OpenStreetMap (OSM).


In [ ]:
# Указываем район (например, Ленинский район Екатеринбурга)
location = "Ленинский район, Екатеринбург"

# Получаем граф уличной сети для района с типом "drive" (автомобильный)
graph = ox.graph_from_place(location, network_type='drive')

# Строим граф
ox.plot_graph(ox.project_graph(graph))

### 1.2 Преобразование графа в GeoDataFrame


Для анализа и визуализации данных можно преобразовать граф в геометрический формат, например, в GeoDataFrame, чтобы работать с линиями и узлами как с геометриями


In [ ]:
# Получаем узлы и ребра в формате GeoDataFrame
nodes, edges = ox.graph_to_gdfs(graph)

# Визуализируем ребра
edges.plot(figsize=(10, 10), color='blue')


## 2. Пространственный сетевой анализ и networkx


### 2.1 Поиск кратчайшего пути между двумя точками


Одним из наиболее распространенных применений сетевого анализа является нахождение кратчайшего пути между двумя точками


In [ ]:
# Указываем стартовую и конечную точки 
start_coords = (60.6572, 56.8385) 
end_coords = (60.6570, 56.8100)    

# Преобразуем координаты в узлы графа
start_node = ox.distance.nearest_nodes(graph, X=start_coords[0], Y=start_coords[1])
end_node = ox.distance.nearest_nodes(graph, X=end_coords[0], Y=end_coords[1])

# Находим кратчайший путь между двумя узлами
route = nx.shortest_path(graph, source=start_node, target=end_node, weight='length')

# Визуализируем маршрут
ox.plot_graph_route(graph, route, route_linewidth=6, node_size=0, bgcolor='k')


Кратчайший путь между POI


In [ ]:
# Загружаем POI (например, рестораны)
pois = ox.geometries_from_place(location, tags={"amenity": "restaurant"})

# Конвертация POI в ту же систему координат, что и граф
pois = pois.to_crs(graph.graph["crs"])

# Выбираем две случайные точки из pois
random_points = pois.sample(n=2, random_state=42) 
poi_1 = random_points.iloc[0].geometry
poi_2 = random_points.iloc[1].geometry

# Шаг 2: Поиск ближайших узлов графа к POI
orig_node = ox.distance.nearest_nodes(graph, X=poi_1.x, Y=poi_1.y)
dest_node = ox.distance.nearest_nodes(graph, X=poi_2.x, Y=poi_2.y)

# Шаг 3: Кратчайший путь
shortest_path = nx.shortest_path(graph, source=orig_node, target=dest_node, weight="length")
shortest_distance = nx.shortest_path_length(graph, source=orig_node, target=dest_node, weight="length")

print("Shortest path:", shortest_path)
print("Shortest distance (meters):", shortest_distance)

# Визуализация
ox.plot_graph_route(graph, shortest_path, route_linewidth=2, node_size=0, bgcolor="white")


### 2.2 Вычисление центральности в узле графов


Центральность узлов в графе помогает определить важность каждого узла в сети.

Существует несколько типов центральности, которые могут быть использованы в зависимости от задачи:

1. Центральность по степени (Degree Centrality) — измеряет количество связей (ребер), которые имеет узел. Узлы с высокой степенью центральности считаются важными, потому что они подключены к большому числу других узлов.

2. Центральность по посредничеству (Betweenness Centrality) — измеряет, насколько часто узел находится на кратчайших путях между другими узлами. Узлы с высокой центральностью между отношениями являются "мостами", которые связывают различные части сети.

3. Центральность по близости (Closeness Centrality) — измеряет, насколько близко узел расположен к остальным узлам в сети. Узлы с высокой центральностью по близости быстро достигают других узлов, что делает их важными для распространения информации.


In [ ]:
# Преобразуем в проекцию UTM для удобства расчетов
graph_projected = ox.project_graph(graph)

# Центральность по степени
degree_centrality = nx.degree_centrality(graph_projected)

# Центральность по посредничеству  (betweenness centrality)
betweenness_centrality = nx.betweenness_centrality(graph_projected, weight='length')

# Центральность по близости
closeness_centrality = nx.closeness_centrality(graph_projected)

Центральность по степени: На карте уличной сети выделяются наиболее "связанные" узлы.


In [ ]:
# Визуализация центральности по степени
fig, ax = plt.subplots(figsize=(10, 10))
node_sizes = [v * 1000 for v in degree_centrality.values()] 

ox.plot_graph(graph_projected, node_size=node_sizes, node_color='red', bgcolor='white', ax=ax)
plt.title("Центральность по степени")

plt.show()

Центральность по посредничеству : Отображение узлов, которые служат "мостами" между различными частями сети.


In [ ]:
# Визуализация центральности по посредничеству 
fig, ax = plt.subplots(figsize=(10, 10))
node_sizes = [v * 1000 for v in betweenness_centrality.values()]

ox.plot_graph(graph_projected, node_size=node_sizes, node_color='blue', bgcolor='white', ax=ax)
plt.title("Центральность по посредничеству ")
plt.show()


Центральность по близости: Выделение узлов, которые могут быстро достигать других узлов в сети.


In [ ]:
# Визуализация центральности по близости
fig, ax = plt.subplots(figsize=(10, 10))
node_sizes = [v * 1000 for v in closeness_centrality.values()]

ox.plot_graph(graph_projected, node_size=node_sizes, node_color='green', bgcolor='white', ax=ax)
plt.title("Центральность по близости")
plt.show()

### 2.3 Нахождение компонент связности в графе

Компоненты связности — это подмножества узлов, которые соединены друг с другом рёбрами, и нет рёбер между компонентами. Этот анализ полезен для изучения сегментации сети.


In [ ]:
# Находим компоненты связности в графе
components = list(nx.connected_components(graph.to_undirected()))

# Выводим количество компонент и размер первой компоненты
print(f"Количество компонент связности: {len(components)}")
print(f"Размер первой компоненты: {len(components[0])}")

- Мы находим компоненты связности в графе, преобразуя его в неориентированный граф.
- Выводим количество компонент и размер первой компоненты.
- Этот анализ помогает понять, насколько сеть разделена на отдельные части.


### 2.4 Вычисление матрицы расстояний между узлами


Матрица расстояний между всеми узлами графа полезна для анализа, чтобы понять, как далеко друг от друга находятся узлы в сети.


In [ ]:
# Вычисляем матрицу расстояний между всеми узлами
distance_matrix = dict(nx.all_pairs_dijkstra_path_length(graph, weight='length'))

# Пример: Расстояние от первого узла до всех других
first_node = list(graph.nodes())[0]
distances_from_first_node = distance_matrix[first_node]

# Выводим первые 5 расстояний
print(list(distances_from_first_node.items())[:5])

- Мы вычисляем матрицу расстояний между всеми узлами с использованием алгоритма Дейкстры.
- Выводим расстояния от первого узла до других узлов в графе.
- Эта информация полезна для анализа доступности и плотности сети.


### 2.5 Изохроны (пешеходная доступность)

Изохроны представляют собой области, которые можно достичь за определенный промежуток времени. Например, можно построить изохрону, которая охватывает территорию, которую можно достичь за 15 минут пешком.


In [ ]:
# Указываем координаты начальной точки (например, центр Ленинского района)
start_coords = (60.6572, 56.8385)

# Преобразуем координаты в узел графа
start_node = ox.distance.nearest_nodes(graph, X=start_coords[0], Y=start_coords[1])

# Определяем максимальный радиус для изохроны (например, 15 минут пешеходного пути)
max_time = 15 * 60  # Время в секундах (15 минут)

# Строим подграф (зону доступности) вокруг ближайшего узла
subgraph = nx.ego_graph(graph, start_node, radius=max_time, distance='length')



In [ ]:
ox.plot_graph(subgraph)

In [ ]:
# Получаем геометрию зоны доступности (линии)
area = ox.convert.graph_to_gdfs(subgraph, nodes=False, edges=True)

# Объединяем все линии в одну геометрию
service_area_lines = area.geometry.unary_union

# Создаем ограничивающую геометрию вокруг участка УДС
isochrone = service_area_lines.convex_hull

# Визуализируем
fig, ax = plt.subplots(figsize=(8, 8))
area.plot(ax=ax, color='lightgray', linewidth=1)
gpd.GeoSeries([isochrone]).plot(ax=ax, color='red', alpha=0.5)
ax.set_title("Isochrone for 15 minutes walk", fontsize=15)
plt.show()



## 3. Пространственный сетевой анализ с использованием внешних API


Множество сервисов и библиотек предоставляют готовые решения для работы с пространственными сетями


### 3.1. [OSRM](https://project-osrm.org)


#### 3.1.1 Кратчайшее расстояние


In [ ]:
import requests

# URL API OSRM
base_url = "http://router.project-osrm.org/route/v1/driving/"

# Координаты точек (долгота, широта)
start_coords = [55.7558, 37.6173]
end_coords = [55.7045, 37.5308]

start = f"{start_coords[1]},{start_coords[0]}" 
end = f"{end_coords[1]},{end_coords[0]}"

# Запрос к API
response = requests.get(f"{base_url}{start};{end}?overview=full")
data = response.json()

# Извлечение расстояния и маршрута
distance = data["routes"][0]["distance"]  # В метрах
route = data["routes"][0]["geometry"]  # GeoJSON

print(f"Shortest distance: {distance / 1000:.2f} km")


Посмотрим на результат на карте


In [ ]:
import polyline 
import folium

# Декодируем маршрут из GeoJSON
decoded_route = polyline.decode(route)

# Визуализация с помощью folium
# Создаем карту с центром в стартовой точке
m = folium.Map(location=start_coords, zoom_start=12, tiles='cartodbpositron')

# Добавляем маршрут на карту
folium.PolyLine(decoded_route, color="blue", weight=5, opacity=0.7).add_to(m)

# Добавляем маркеры для стартовой и конечной точки
folium.Marker(start_coords, popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(end_coords, popup="End", icon=folium.Icon(color="red")).add_to(m)

# Отображаем карту
m

### 3.2. [Open Route Service](https://openrouteservice.org)


#### 3.2.1. Изохроны


In [ ]:
import requests

# API OpenRouteService (требуется ключ API)
ors_api_key = "your_key"
url = "https://api.openrouteservice.org/v2/isochrones/foot-walking"

# Параметры
params = {
    "locations": [[37.6173, 55.7558]], 
    "range": [300, 600, 900],  # Изохроны в секундах (5, 10, 15 минут)
}

headers = {
    "Authorization": ors_api_key,
    "Content-Type": "application/json",
}

# Запрос к API
response = requests.post(url, json=params, headers=headers)
isochrones = response.json()



In [ ]:

# Преобразуем GeoJSON в GeoDataFrame
gdf_isochrones = gpd.GeoDataFrame.from_features(
    isochrones["features"], crs="EPSG:4326"
)


#Визуализируем результат на карте
gdf_isochrones.explore( tiles='cartodbpositron')


#### 3.2.2. Кратчайшее расстояние


In [ ]:
ors_url = "https://api.openrouteservice.org/v2/directions/driving-car"


# Параметры запроса
params = {
    "start": "37.6173,55.7558",
    "end": "37.5665,55.7332",
}

headers = {
    "Authorization": ors_api_key,
}

response = requests.get(ors_url, params=params, headers=headers)
path = response.json()

# Достаем расстояние и время
distance = path["features"][0]["properties"]["segments"][0]["distance"]  # в метрах
duration = path["features"][0]["properties"]["segments"][0]["duration"]  # в секундах

print(f"Distance: {distance / 1000:.2f} km")
print(f"Duration: {duration / 60:.2f} minutes")


# Преобразуем GeoJSON в GeoDataFrame
gdf_path = gpd.GeoDataFrame.from_features(
    path["features"], crs="EPSG:4326"
)


#Визуализируем результат на карте
gdf_path.explore(tiles='cartodbpositron')


#### 3.2.3 Матрица расстояний


In [ ]:
ors_matrix_url = "https://api.openrouteservice.org/v2/matrix/driving-car"

# Координаты
locations = [
    [37.6173, 55.7558], 
    [37.5665, 55.7332],  
    [37.5905, 55.7602],
]

data = {
    "locations": locations,
    "metrics": ["distance", "duration"],
}

headers = {
    "Authorization": ors_api_key,
    "Content-Type": "application/json",
}

response = requests.post(ors_matrix_url, json=data, headers=headers)
matrix = response.json()

# Вывод матрицы расстояний
print(matrix["distances"])
print(matrix["durations"])


### 3.3 [Graphopper](https://www.graphhopper.com)


#### 3.3.1. Кратчайшее расстояние


In [ ]:

gh_url = "https://graphhopper.com/api/1/route"
graphhopper_api_key = "your_key"

# Параметры запроса
params = {
    "point": ["55.7558,37.6173", "55.7332,37.5665"],  # Москва, две точки
    "vehicle": "car",
    "key": graphhopper_api_key,
}

response = requests.get(gh_url, params=params)
route = response.json()

# Достаем расстояние и время
distance = route["paths"][0]["distance"]  # в метрах
duration = route["paths"][0]["time"] / 1000  # в секундах

print(f"Distance: {distance / 1000:.2f} km")
print(f"Duration: {duration / 60:.2f} minutes")




#### 3.3.2 Матрица расстояний


In [ ]:
gh_matrix_url = "https://graphhopper.com/api/1/matrix"
headers = {"Content-Type": "application/json"}

# Координаты
locations = [
    [37.6173, 55.7558],  # точка 1
    [37.5665, 55.7332],  # точка 2
    [37.5905, 55.7602],  # точка 3
]

data = {
    "points": locations,
    "vehicle": "car",
    "key": graphhopper_api_key,
}


response = requests.post(gh_matrix_url, json=data, headers=headers)
matrix = response.json()



## 3.Создание геометрий кварталов на основе улично-дорожной сети


Подготовка дополнительных данных - выгружаем из OSM


In [ ]:
location = "Екатеринбург, Россия"

#Границы района
area = ox.geocode_to_gdf(location)
#определеяем UTM зону
target_crs = area.estimate_utm_crs()
#перепроецируем данные
area_utm = area.to_crs(target_crs)


#Здания
buildings = ox.features_from_place(location, {'building': True}  ) 
#Обрабатываем данные
building_utm = buildings.to_crs(target_crs)
required_columns = ['name', 'official_name', 'operator:type', 'geometry']
building_utm = building_utm[required_columns]
building_utm = building_utm.loc[building_utm.geom_type.isin(['Polygon', 'MultiPolygon'])]
building_utm = building_utm.reset_index()

#Вода
rivers = ox.geometries_from_place(location, tags={'natural': 'water'})
#Обрабатываем данные
rivers = rivers.to_crs(target_crs)
rivers  = rivers.loc[rivers.geom_type.isin(['Polygon', 'MultiPolygon'])]


#Парки
green_zones = ox.geometries_from_place(
    location, 
    tags={
        "leisure": ["park", "garden", "recreation_ground", "nature_reserve"],
        "landuse": ["forest", "grass", "meadow", "orchard"],
        "natural": ["wood", "grassland", "wetland"]
    }
)
green_zones = green_zones.loc[green_zones.geom_type.isin(['Polygon', 'MultiPolygon'])]
green_zones_utm = green_zones.to_crs(target_crs)


#Улично-дорожная сеть
graph = ox.graph_from_place(location, network_type="drive")
edges = ox.graph_to_gdfs(graph, nodes=False, edges=True) #конвертируем ребра графа в GeoDataFrame
edges = edges.to_crs(target_crs)



Создание полигонов - кварталов


In [ ]:
# Строим буфер для каждой дороги
buffered_edges = edges.geometry.buffer(15)  # Радиус буфера в метрах

# Объединеняем буферы
buffered_union = gpd.GeoSeries(buffered_edges).unary_union
buffered_roads = gpd.GeoDataFrame(geometry=[buffered_union], crs=target_crs)

buffered_roads.explore( tiles='cartodbpositron')

# # Посмотрим, что получилось
# buffered_roads.plot(color="lightblue", edgecolor="black", figsize=(10, 10))

Вырезаем кварталы из городской территории


In [ ]:
city_blocks = gpd.overlay(buffered_roads, area_utm, how='symmetric_difference')
city_blocks.explore(tiles='cartodbpositron')


Разделяем мультиполигон на полигоны


In [ ]:
split_city_blocks = city_blocks.explode(index_parts=False)
split_city_blocks.explore(tiles='cartodbpositron')

Есть проблема, что кварталы, которые находятся по разную сторону рек или парка относятся к одному полигону. Это можно решить "вырезанием" из кварталов воды и зеленых зон. Это мы сможем сделать с помощью Symmetric Difference


In [ ]:
#Отфильтруем реки и зелные зоны, чтобы только большие объекты делили наши кварталы
min_area = 5000
filtered_green_zones = green_zones_utm[green_zones_utm.geometry.area > min_area]
filtered_rivers = rivers[rivers.geometry.area > min_area]

combined = filtered_green_zones.append(filtered_rivers, ignore_index=True).geometry.unary_union
combined_gdf = gpd.GeoDataFrame(geometry=[combined], crs=filtered_green_zones.crs)


#Вырезаем
split_blocks_2 = gpd.overlay(city_blocks, combined_gdf, how='symmetric_difference')
# split_blocks_3 = gpd.overlay(city_blocks, filtered_green_zones, how='symmetric_difference')


#Разделяем
split_city_blocks = split_blocks_2.explode(index_parts=False)
split_city_blocks.explore(tiles='cartodbpositron')

Есть проблема, что часть кварталов - не кварталы вовсе, а какие-то мелкие обрезки. Другие - поля непонятные. Давайте оставим только те участки, где есть здания


In [ ]:
#Выполним пространственное объединение кварталов и зданий
blocks_with_buildings = gpd.sjoin(split_city_blocks, building_utm, how="inner", op="intersects")

#Удалим дубликаты
blocks_with_buildings = blocks_with_buildings.drop_duplicates(subset='geometry')

#Посмотрим на результат
blocks_with_buildings.explore(tiles='cartodbpositron')

## 4.Работа с растровыми данными на примере данных о населении (World Pop)


Экспортируем данные о населении для всей России (опционально, набор для работы уже лежит в директории)


In [ ]:
# Ссылка на данные WorldPop
url = "https://data.worldpop.org/GIS/Population/Global_2000_2020_Constrained/2020/BSGM/RUS/rus_ppp_2020_constrained.tif"

# Имя файла для сохранения
output_file = "worldpop_russia_2020.tif"

# Скачивание файла
response = requests.get(url, stream=True)

if response.status_code == 200:
    with open(output_file, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print(f"File saved as {output_file}")
else:
    print("Failed to download file. Status code:", response.status_code)


Открываем скачанный файл


In [ ]:
import rasterio

with rasterio.open(output_file) as dataset:
    print("CRS:", dataset.crs)
    print("Bounds:", dataset.bounds)
    print("Resolution:", dataset.res)


Обрезаем данные о численности населения по границе города (area - границы вашего района в СRS - WGS84)


In [ ]:
from rasterio.mask import mask
from shapely.geometry import mapping

# Загрузка векторных данных
# city_gdf = gpd.read_file("city_boundary.geojson")
city_geometry = [mapping(area.geometry.unary_union)]

# Вырезание только bbox для более компактного растра
with rasterio.open(output_file) as src:
    # Вычисление обрезки по bbox
    bbox = area.total_bounds  # [minx, miny, maxx, maxy]
    window = src.window(*bbox)
    
    # Читаем данные в окне
    data = src.read(window=window)
    cropped_transform = src.window_transform(window)
    profile = src.profile

    profile.update({
        "height": data.shape[1],
        "width": data.shape[2],
        "transform": cropped_transform
    })
    
    # Сохраняем уменьшенный растр
    with rasterio.open("cropped_population.tif", "w", **profile) as dst:
        dst.write(data)
    

Получаем информацию о сохраненных данных


In [ ]:
# Открытие файла
file_path = "final_population.tif"
with rasterio.open(file_path) as src:
    # Вывод основной информации о растре
    print("CRS:", src.crs)  # Система координат
    print("Bounds:", src.bounds)  # Границы растра
    print("Width, Height:", src.width, src.height)  # Размер растра
    print("Number of bands:", src.count)  # Количество слоев
    print("Data type:", src.dtypes)  # Тип данных
    print("Transform:", src.transform)  # Аффинная трансформация


Получаем информацию о размере одного пикселя


In [ ]:
with rasterio.open(file_path) as src:
    transform = src.transform  # Аффинная трансформация растра

    # Извлечение размеров пикселя
    pixel_width = transform.a  # Размер пикселя по оси X
    pixel_height = -transform.e  # Размер пикселя по оси Y (берем с минусом, так как Y направлен вниз в системе координат)

    print(f"Pixel Width: {pixel_width} units")
    print(f"Pixel Height: {pixel_height} units")


Перепроецируем растр в UTM


In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

output_file = "worldpop_reprojected.tif"

with rasterio.open(file_path) as src:
    transform, width, height = calculate_default_transform(src.crs, target_crs, src.width, src.height, *src.bounds)
    profile = src.profile
    profile.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(output_file, 'w', **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )


И еще раз посмотрим на информацию о размере одного пикселя


In [ ]:
with rasterio.open(output_file) as src:
    transform = src.transform 

    # Извлечение размеров пикселя
    pixel_width = transform.a  # Размер пикселя по оси X
    pixel_height = -transform.e  # Размер пикселя по оси Y (берем с минусом, так как Y направлен вниз в системе координат)

    print(f"Pixel Width: {pixel_width} units")
    print(f"Pixel Height: {pixel_height} units")


Извлечение данных из растра


In [ ]:
with rasterio.open(file_path) as src:
    # Считываем первый слой (или единственный, если это однобандовый растр)
    data = src.read(1)  # Чтение 1-го слоя

    # Выводим статистику
    print("Min value:", np.min(data))
    print("Max value:", np.max(data))
    print("Mean value:", np.mean(data))


In [ ]:
import matplotlib.pyplot as plt

with rasterio.open(output_file) as src:
    data = src.read(1)  # Чтение 1-го слоя

    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='viridis')  # Визуализация с использованием цветовой карты
    plt.colorbar(label="Population Density")  # Добавляем цветовую шкалу
    plt.title("Population Density (WorldPop, Russia, 2020)")
    plt.show()


## 5.Расчет плотности населения по кварталам на основе данных World Pop


Зональная статистика


In [ ]:
import rasterstats as rs

# Рассчитываем зональную статистику (сумма значений растра для каждого полигона)
stats = rs.zonal_stats(blocks_with_buildings, output_file, stats="sum", geojson_out=True)

# Преобразуем результат в GeoDataFrame
gdf_stats = gpd.GeoDataFrame.from_features(stats)

# Теперь у вас есть GeoDataFrame с дополнительной колонкой, содержащей сумму значений для каждого полигона
gdf_stats.head()


In [ ]:
gdf_stats = gdf_stats.set_crs(target_crs)

In [ ]:
gdf_stats.explore(tiles='cartodbpositron')

Вычисляем плотность населения


In [ ]:
gdf_stats['density'] = gdf_stats['sum']/(gdf_stats.geometry.area/1000000)

Смотрим на результат


In [ ]:
gdf_stats.explore(column='density')